In [2]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=0494a0c9a7092da45efa9110afda1a2d98dd231af3a2eb9ab5a8b77f978cff08
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=bb079f204b976642a4891d9a6915b359c69b68b9a40a8f2f778a9ebc8814cf6d
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.3.0


In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
#scale down the pixels to 0-255 to make faster evaluation
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model
  

In [11]:
from kerastuner import RandomSearch #this random search will help which parameter will be best
from kerastuner.engine.hyperparameters import HyperParameters #

In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5246 - accuracy: 0.8161 - val_loss: 0.3864 - val_accuracy: 0.8515
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3762 - accuracy: 0.8614 - val_loss: 0.3783 - val_accuracy: 0.8600
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3494 - accuracy: 0.8715 - val_loss: 0.3832 - val_accuracy: 0.8618


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4974 - accuracy: 0.8219 - val_loss: 0.4447 - val_accuracy: 0.8330
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3788 - accuracy: 0.8621 - val_loss: 0.3725 - val_accuracy: 0.8635
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3441 - accuracy: 0.8753 - val_loss: 0.3873 - val_accuracy: 0.8552


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4158 - accuracy: 0.8493 - val_loss: 0.3051 - val_accuracy: 0.8895
Epoch 2/3
1688/1688 [==============================] - 8s 4ms/step - loss: 0.2765 - accuracy: 0.8982 - val_loss: 0.2765 - val_accuracy: 0.8997
Epoch 3/3
1688/1688 [==============================] - 8s 4ms/step - loss: 0.2245 - accuracy: 0.9156 - val_loss: 0.2660 - val_accuracy: 0.9013


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4289 - accuracy: 0.8466 - val_loss: 0.3147 - val_accuracy: 0.8847
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2817 - accuracy: 0.8972 - val_loss: 0.2727 - val_accuracy: 0.9012
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2308 - accuracy: 0.9141 - val_loss: 0.2753 - val_accuracy: 0.9037


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3810 - accuracy: 0.8627 - val_loss: 0.2931 - val_accuracy: 0.8945
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2427 - accuracy: 0.9105 - val_loss: 0.2463 - val_accuracy: 0.9118
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1774 - accuracy: 0.9340 - val_loss: 0.2414 - val_accuracy: 0.9163


INFO:tensorflow:Oracle triggered exit


In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        41520     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1769536   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,812,666
Trainable params: 1,812,666
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1281 - accuracy: 0.9525 - val_loss: 0.2564 - val_accuracy: 0.9142
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0891 - accuracy: 0.9671 - val_loss: 0.3149 - val_accuracy: 0.9163
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0632 - accuracy: 0.9763 - val_loss: 0.3200 - val_accuracy: 0.9157
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0445 - accuracy: 0.9837 - val_loss: 0.3949 - val_accuracy: 0.9112
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0338 - accuracy: 0.9876 - val_loss: 0.4393 - val_accuracy: 0.9095
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0271 - accuracy: 0.9906 - val_loss: 0.4679 - val_accuracy: 0.9112
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0245 - accuracy: 0.9914 - val_loss: 0.5406 - val_accuracy